In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
from matplotlib.patches import Circle
import scipy.signal as sig

N = 9  # order of recursive filter


def zplane(z, p, title="Poles and Zeros"):
    "Plots zero and pole locations in the complex z-plane"
    ax = plt.gca()

    ax.plot(np.real(z), np.imag(z), "bo", fillstyle="none", ms=10)
    ax.plot(np.real(p), np.imag(p), "rx", fillstyle="none", ms=10)
    unit_circle = Circle(
        (0, 0), radius=1, fill=False, color="black", ls="solid", alpha=0.9
    )
    ax.add_patch(unit_circle)
    ax.axvline(0, color="0.7")
    ax.axhline(0, color="0.7")

    plt.title(title)
    plt.xlabel(r"Re{$z$}")
    plt.ylabel(r"Im{$z$}")
    plt.axis("equal")
    plt.xlim((-2, 2))
    plt.ylim((-2, 2))
    plt.grid()


# design filter
b, a = sig.butter(N, 0.2)
# decomposition into SOS
sos = sig.tf2sos(b, a, pairing="nearest")


# print filter coefficients
print("Coefficients of the recursive part \n")
print(["%1.2f" % ai for ai in a])
print("\n")
print("Coefficients of the recursive part of the individual SOS \n")
print("Section \t a1 \t\t a2")
for n in range(sos.shape[0]):
    print("%d \t\t %1.5f \t %1.5f" % (n, sos[n, 4], sos[n, 5]))

# plot pole and zero locations
plt.figure(figsize=(5, 5))
zplane(np.roots(b), np.roots(a), "Poles and Zeros - Overall")

plt.figure(figsize=(10, 7))
for n in range(sos.shape[0]):
    plt.subplot(231 + n)
    zplane(
        np.roots(sos[n, 0:3]),
        np.roots(sos[n, 3:6]),
        title="Poles and Zeros - Section %d" % n,
    )
plt.tight_layout()

# compute and plot frequency response of sections
plt.figure(figsize=(10, 5))
for n in range(sos.shape[0]):
    Om, H = sig.freqz(sos[n, 0:3], sos[n, 3:6])
    plt.plot(Om, 20 * np.log10(np.abs(H)), label=r"Section %d" % n)

plt.xlabel(r"$\Omega$")
plt.ylabel(r"$|H_n(e^{j \Omega})|$ in dB")
plt.legend()
plt.grid()
